In [24]:
import pickle
import numpy
from music21 import instrument, note, stream, chord
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.layers import BatchNormalization as BatchNorm
from keras.layers import Activation

In [9]:
def generate():
    with open('data/notes', 'rb') as filepath:
        notes = pickle.load(filepath)
        
        pitchnames = sorted(set(item for item in notes))
        n_vocab = len(set(notes))
        
        network_ip, normalized_ip = prepare_sequences(notes, pitchnames, n_vocab)
        model = create_network(normalized_ip, n_vocab)
        prediction_op = generate_notes(model, network_ip, pitchnames, n_vocab)
        create_midi(prediction_op)

In [15]:
def prepare_sequences(notes, pitchnames, n_vocab):
    note_to_int = dict((note, number) for number, note in enumerate(pitchnames))
    
    sequence_length = 100
    network_ip = []
    output = []
    for i in range(0, len(notes)-sequence_length, 1):
        sequence_in = notes[i:i + sequence_length]
        sequence_op = notes[i + sequence_length]
        network_ip.append([note_to_int[char] for char in sequence_in])
        output.append(note_to_int[sequence_op])
        
    n_patterns = len(network_ip)
    normalized_ip = numpy.reshape(network_ip, (n_patterns, sequence_length,1))
    normalized_ip = normalized_ip/ float(n_vocab)
    
    return (network_ip,  normalized_ip)

In [17]:
def create_network(network_ip, n_vocab):
    model = Sequential()
    model.add(LSTM(
        512,
        input_shape=(network_ip.shape[1], network_ip.shape[2]),
        recurrent_dropout=0.3,
        return_sequences=True
    ))
    model.add(LSTM(512, return_sequences=True, recurrent_dropout=0.3,))
    model.add(LSTM(512))
    model.add(BatchNorm())
    model.add(Dropout(0.3))
    model.add(Dense(256))
    model.add(Activation('relu'))
    model.add(BatchNorm())
    model.add(Dropout(0.3))
    model.add(Dense(n_vocab))
    model.add(Activation('softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='rmsprop')

    # Load the weights to each node
    model.load_weights('weights.hdf5')

    return model

In [4]:
def create_midi(prediction_op):
    
    for pattern in prediction_op:
        if('.' in pattern)or pattern.isdigit():
            notes_in_chord = pattern.split('.')
            notes = []
            for current_note in notes_in_chord:
                new_note = note.Note(int(current_note))
                new_note.storedInstrument = instrument.Piano()
                notes.append(new_note)
            new_chord = chord.Chord(notes)
            new_chord.offset = offset
            output_notes.append(new_chord)
        else:
            new_note = note.Note(pattern)
            new_note.offset = offset
            new_note.storedInstrument = instrument.Piano()
            output_notes.append(new_note)
            
        offset += 0.5
        
    midi_stream = stream.Stream(output_notes)
    midi_stream.write('midi', fp='test_op'.mid)

In [27]:
if __name__ == '__main__':
    generate()

OSError: Unable to open file (unable to open file: name = 'weights.hdf5', errno = 2, error message = 'No such file or directory', flags = 0, o_flags = 0)